# Chainer mnist tutorial2

自分の理解が深まるように少し改変  
http://docs.chainer.org/en/stable/tutorial/basic.html  
そしてChainerに付属のReporterを使えるようにした

# Run all
GPU: 0  
unit: 1000  
Minibatch-size: 100  
epoch: 20  

In [1]:
#!/usr/bin/env python
from __future__ import print_function
import argparse
import sys; sys.argv=['']; del sys

import numpy as np
import chainer
import chainer.functions as F
import chainer.links as L
from chainer import training
from chainer.training import extensions
from chainer import Variable
import copy # test iterator copy

In [2]:
class MLP(chainer.Chain):
    def __init__(self, n_units, n_out):
        super(MLP, self).__init__(
             # the size of the inputs to each layer will be inferred
            l1=L.Linear(784, n_units),  # n_in -> n_units
            l2=L.Linear(n_units, n_units),  # n_units -> n_units
            l3=L.Linear(n_units, n_out),    # n_units -> n_out
            )
    def __call__(self, x):
        h1 = F.relu(self.l1(x))
        h2 = F.relu(self.l2(h1))
        y = self.l3(h2)
        return y

In [3]:
class Classifier(chainer.Chain):
    def __init__(self, predictor):
        super(Classifier, self).__init__(predictor=predictor)
        self.accuracy = None
    def __call__(self, x, t):
        y = self.predictor(x)
        loss = F.softmax_cross_entropy(y, t)
        accuracy = F.accuracy(y, t)
        chainer.report({'loss': loss, 'accuracy': accuracy}, self)
        return loss


In [16]:
class MyUpdater(chainer.training.StandardUpdater):

    def __init__(self, iterator, optimizer, 
                 converter=chainer.dataset.convert.concat_examples, device=None):
        if isinstance(iterator, chainer.dataset.iterator.Iterator):
            iterator = {'main':iterator}

        self._iterators = iterator

        if isinstance(optimizer, chainer.optimizer.Optimizer):
            optimizer = {'main': optimizer}
        self._optimizers = optimizer

        self.converter = converter
        self.device = device
        self.iteration = 0

    def update_core(self):
        batch = self._iterators['main'].next()
        in_arrays = self.converter(batch, self.device)

        optimizer = self._optimizers['main']
        
        loss = optimizer.target(in_arrays[0], in_arrays[1])
        optimizer.target.cleargrads()
        loss.backward()
        optimizer.update()

In [17]:
unitsize = 1000
batchsize = 100
epochsize = 10

In [18]:
gpu_id = 0 #  use gpu
model = Classifier(MLP(unitsize, 10))

if gpu_id >= 0:
    chainer.cuda.get_device(gpu_id).use()  # Make a specified GPU current
    model.to_gpu()  # Copy the model to the GPU

In [27]:
# optimizer
# optimizer = chainer.optimizers.SGD()
optimizer = chainer.optimizers.Adam()

optimizer.use_cleargrads()
optimizer.setup(model)

In [28]:
# Load the MNIST dataset
train, test = chainer.datasets.get_mnist()
train_iter = chainer.iterators.SerialIterator(train, batchsize)
test_iter = chainer.iterators.SerialIterator(test, batchsize, repeat=False, shuffle=False)
testsize = len(test)
testsize

10000

In [29]:
trainsize = len(train)
trainsize

60000

In [30]:
# Set up a trainer
updater = MyUpdater(train_iter, optimizer, device=gpu_id)
trainer = training.Trainer(updater, (epochsize, 'epoch'), out='result')

In [31]:
# Dump a computational graph from 'loss' variable at the first iteration
# The "main" refers to the target link of the "main" optimizer.
trainer.extend(extensions.dump_graph('main/loss'))

# Take a snapshot at each epoch
trainer.extend(extensions.snapshot(), trigger=(10, 'epoch'))
trainer.extend(extensions.Evaluator(test_iter, model, device=gpu_id))
# Write a log of evaluation statistics for each epoch
trainer.extend(extensions.LogReport())
trainer.extend(extensions.PrintReport(
    ['epoch', 'main/loss', 'validation/main/loss',
     'main/accuracy', 'validation/main/accuracy', 'elapsed_time']))

# Print a progress bar to stdout
interval = trainsize/batchsize/2
trainer.extend(extensions.ProgressBar(update_interval=interval))

In [32]:
# Run the training
trainer.run()

epoch       main/loss   validation/main/loss  main/accuracy  validation/main/accuracy  elapsed_time
     total [##................................................]  5.00%
this epoch [#########################.........................] 50.00%
       300 iter, 0 epoch / 10 epochs
       inf iters/sec. Estimated time to finish: 0:00:00.
1           0.0193066   0.101901              0.994599       0.9794                    2.39655       
     total [#####.............................................] 10.00%
this epoch [..................................................]  0.00%
       600 iter, 1 epoch / 10 epochs
    232.61 iters/sec. Estimated time to finish: 0:00:23.214352.
     total [#######...........................................] 15.00%
this epoch [#########################.........................] 50.00%
       900 iter, 1 epoch / 10 epochs
    249.15 iters/sec. Estimated time to finish: 0:00:20.469658.
2           0.0124841   0.0848928             0.996232       0.9839         